In [ ]:
import os
import sys
# os.chdir('/Users/ludo/Desktop/animalai/animalai')
sys.path.insert(0, "/Users/ludo/Desktop/animalai/animalai/animalai_train")
sys.path.insert(1, "/Users/ludo/Desktop/animalai/animalai/animalai")
from animalai.envs.gym.environment import AnimalAIGym
import random
from animalai.envs.arena_config import ArenaConfig

import yaml
import copy
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt


# CONFIG
env_path = '../../env/aaiagain'
worker_id = 3
#random.randint(1, 100)
# competition_envs = os.listdir('../competition_configurations/')

seed = 1


# yaml_path = '../examples/rand_env.yml'
# ac = ArenaConfig(yaml_path)
# arena_path = "../../competition_configurations/5-10-3.yml" #'configurations/curriculum/0.yaml'
# arena_path = '../../competition_configurations/9-5-1.yml'
arena_path = 'test.yml'
# arena_path = '../configurations/ramps/0.yml'

# ac = ArenaConfig(arena_path)
# ac.arenas[1] = copy.deepcopy(ac.arenas[0])
# yaml_path = 'configurations/curriculum4/19.yml'
ac = ArenaConfig(arena_path)
env = AnimalAIGym(environment_filename=env_path,
              worker_id=worker_id,
              n_arenas=1,
              arenas_configurations=ac,
                use_visual=False,
                seed=seed,
                 grayscale=False,
                 resolution=1000)

x = env.step([[0,0]])

In [ ]:
x = env.reset()
plt.imshow(env.render())

In [ ]:

for i in range(10):
    env.step([[0,0]])
    plt.imshow(env.render())

In [ ]:
from animalai.envs.ramp_creator import run as rc
# print(rc()[0].replace('\t', ''))
# from collections import namedtuple as nt
# from random import randrange, choice
# def make_obj(pos=False, size=False, name=False, rot=False):
#     res =  f"""
#     - !Item
#       name: {name}"""
#     if pos:
#         res +=f"""
#       positions:
#       - !Vector3 {{x: {pos.x} , y: {pos.y}, z: {pos.z}}}"""
#     if size:
#         res +=f"""
#       sizes:
#       - !Vector3 {{x: {size.x}, y: {size.y}, z: {size.z}}}"""
#     if rot:
#         res += f"""
#       rotations: [{rot}]"""
#     return res


# def run():
#     vector = nt('vec', ['x', 'y', 'z'])
#     # agent_p = vector(randrange(1,39), 0 , randrange(1,39), randrange(1,39))
#     ramp_p = vector(randrange(10,30),0, randrange(10,30))
#     ramp_width = randrange(2,ramp_p.x)
#     ramp_length = randrange(2,ramp_p.z)
#     ramp_height = randrange(2,max(int(ramp_length*(3/5)),3))
#     ramp_s = vector(ramp_width, ramp_height, ramp_length)
#     rot = 180


#     wall_p = vector(ramp_p.x, 0, 1.5 + ramp_p.z+ramp_s.z/2)
#     wall_s = vector(ramp_width, ramp_s.y, 3)
#     goodgoal_p = vector(wall_p.x, 7, wall_p.z)
#     goodgoal_s = vector(3,3,3)
#     base = """
# !ArenaConfig
# arenas:
#   0: !Arena
#     pass_mark: 2
#     t: 250
#     items:"""
#     for obj in ['Agent', 'Ramp', 'Wall']:
#         inp = {"name":obj}
#         if obj.lower()+'_p' in locals():
#             inp['pos'] = locals()[obj.lower()+'_p']
#         if obj.lower()+'_s' in locals():
#             inp['size'] = locals()[obj.lower()+'_s']
#         if obj not in ['Agent', 'GoodGoal']:
#             inp['rot'] = rot
#         base+=make_obj(**inp)

#     for i in range(int(ramp_width/2)):
#         inp = {
#             "name": "GoodGoal",
#             "pos": vector(wall_p.x-wall_s.x/2+2*i, 7, wall_p.z),
#             "size": vector(2,2,2)
#         }
#         base+=make_obj(**inp)
#     return [base]
yaml.load(rc()[0], Loader=yaml.Loader)


In [ ]:
base = """!ArenaConfig
arenas:
  0: !Arena
    pass_mark: 2
    t: 250
    items:
    - !Item
      name: Agent
    - !Item
      name: Ramp
      positions:
      - !Vector3 {x: 15 , y: 0, z: 12}
      sizes:
      - !Vector3 {x: 8, y: 2, z: 7}
      rotations: [180]
    - !Item
      name: Wall
      positions:
      - !Vector3 {x: 15 , y: 0, z: 17.0}
      sizes:
      - !Vector3 {x: 8, y: 2, z: 3}
      rotations: [180]
    - !Item
      name: GoodGoal
      positions:
      - !Vector3 {x: 11.0 , y: 7, z: 17.0}
      sizes:
      - !Vector3 {x: 2, y: 2, z: 2}
    - !Item
      name: GoodGoal
      positions:
      - !Vector3 {x: 13.0 , y: 7, z: 17.0}
      sizes:
      - !Vector3 {x: 2, y: 2, z: 2}
    - !Item
      name: GoodGoal
      positions:
      - !Vector3 {x: 15.0 , y: 7, z: 17.0}
      sizes:
      - !Vector3 {x: 2, y: 2, z: 2}
    - !Item
      name: GoodGoal
      positions:
      - !Vector3 {x: 17.0 , y: 7, z: 17.0}
      sizes:
      - !Vector3 {x: 2, y: 2, z: 2}"""

In [ ]:
base

In [ ]:
import yaml
# yaml.load(open(arena_path, "r"), Loader=yaml.Loader).arenas
x = yaml.load(base, Loader=yaml.Loader)
# ArenaConfig([base])
x

In [ ]:
env.reset()
for i in range(30):
    x = env.step([[0,0]])


In [ ]:
# x[3]['batched_step_result'].obs[0]
plt.imshow(env.render())


In [ ]:
from copy import deepcopy
ac = deepcopy(ArenaConfig(arena_path))
ac.arenas = {k:v for k,v in ac.arenas.items() if k==1}
ac.arenas

In [ ]:
env._env.rl_output.agentInfos["AnimalAI?team=0"].value[0].reward

In [ ]:
os.chdir('..')
import time
from config import COMPETITION_CONFIGS as comp
for i in comp['Numerosity']:
    print(i)
    yaml_path = f'../competition_configurations/{i}.yml'
    ac = ArenaConfig(yaml_path)    
    env.reset(ac)
    for i in range(30):
        env.step([0,0])
    time.sleep(1)

In [ ]:
env.close()

In [ ]:
x[3]['batched_step_result'].reward


In [ ]:
from animalai.envs.cvis_test import ExtractFeatures
ef = ExtractFeatures(display=True, training=False)
y = ef.run_test(env.render())
plt.imshow(ef.img[:,:,::-1])
plt.savefig('/Users/ludo/Desktop/rand_env.png',bbox_inches='tight',transparent=True, pad_inches=0)


In [ ]:
def goal_over(bb1, bb2):
    if 'goal' in bb2[1]:
        return False
    if get_distance(bb1[0], bb2[0])<0.005:
        print('ye')

        if bb1[0][1]<bb2[0][1]:
            print('naa')
            return True
    return False
goal_over(y['goal1'][2], y['platform'][0])

In [ ]:
from animalai.envs.cvis import ExtractFeatures
ef = ExtractFeatures(display=True, training=False)
data = x[3]['batched_step_result'].obs[0][0]
print(ef.run(data))
plt.imshow(ef.img)

In [ ]:
class HSV:
	"""Color lower and upper bounds in HSV format"""

	@classmethod
	def __getattribute__(cls,attr):
		return [np.array(i) for i in getattr(cls, attr)]

	green = [[33,80,40], [102,255,255]]
	# red = [[0,181,0], [6,255,255]]
	# red = [[0,184,0], [179,208,217]]
	red = [[125,16,88], [179,255,255]]
	grey = [[0,0,0], [0,0,224]]
	brown = [[7,53,40], [18,87,121]]

hsv_cls = HSV()
objects = {
	'goal': hsv_cls.green,
	# 'danger_zone': hsv_cls.red,
	# 'wall': hsv_cls.grey,
}
ef.get_contour(objects['goal'])
plt.imshow(ef.hsv_img)

In [ ]:
# env._env.DEFAULT_EDITOR_PORT
x[3]['batched_step_result'].obs[1].reshape(-1,9)[[0,11,22]][:, 0:6].flatten()

In [ ]:
env.close()

In [ ]:
env.reset()
for _ in range(1):
    x = env.step([[0,0]])
    res = x[3]['batched_step_result'].obs[1][0]
    plt.axis('off')
    plt.imshow(res)
#     plt.savefig('rand_env.png',bbox_inches='tight',transparent=True, pad_inches=0)
    plt.show()
    res = x[3]['batched_step_result'].obs[1][1]
    plt.axis('off')
    plt.imshow(res)
#     plt.savefig('rand_env.png',bbox_inches='tight',transparent=True, pad_inches=0)
    plt.show()    

In [ ]:
import sys
sys.path.insert(0, "/Users/ludo/Desktop/animalai/animalai/animalai_train")
sys.path.insert(1, "/Users/ludo/Desktop/animalai/animalai/animalai")
from animalai.envs.gym.environment import AnimalAIGym
import random
from animalai.envs.arena_config import ArenaConfig

import os
import yaml
import copy
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from animalai.envs.environment import AnimalAIEnvironment

# CONFIG
env_path = '../../env/AnimalAI'
# env_path = '../env/AnimalAI'

worker_id = 2
seed = 1
# yaml_path = '../configurations/ramps/0.yml'

# yaml_path = '../../competition_configurations/3-18-3.yml'
# yaml_path = '../configurations/curriculum12/112.yml'
# yaml_path = "../../competition_configurations/5-8-3.yml" #'configurations/curriculum/0.yaml'
yaml_path = 'test.yml'

ac = ArenaConfig(yaml_path)

env = AnimalAIEnvironment(
                    file_name=env_path,
                    arenas_configurations = ac,
                    worker_id=worker_id,
                    play=True, base_port=5002,
                    alter_obs = False
                    )
# env = AnimalAIGym(environment_filename=env_path,
#               worker_id=worker_id,
#               n_arenas=1,
#               arenas_configurations=ac,

#                 seed=seed,
#                  grayscale=False)
x = env.step()

# plt.axis('off')
# plt.imshow(env.visual_obs)
# plt.savefig('rand_env.png',bbox_inches='tight',transparent=True, pad_inches=0)


In [ ]:
env.close()

In [ ]:
plt.imshow(env.get_step_result('AnimalAI?team=0').obs[0][0])


In [ ]:
from copy import deepcopy

# yaml_path = '../../competition_configurations/3-18-3.yml'
# yaml_path = '../configurations/curriculum12/112.yml'
yaml_path = "../../competition_configurations/5-8-3.yml" #'configurations/curriculum/0.yaml'
# yaml_path = 'test.yml'

ac = deepcopy(ArenaConfig(yaml_path))
# ac.arenas = {0:v for k,v in ac.arenas.items() if k==2}


In [ ]:
arenas_configurations_proto = ac.to_proto()
arenas_configurations_proto_string = arenas_configurations_proto.SerializeToString(
    deterministic=True
)
env.arenas_parameters_side_channel.send_raw_data(
    bytearray(arenas_configurations_proto_string)
)


In [ ]:
env.reset()

In [ ]:
ac.arenas[2].

In [ ]:
0 is ascending towards bottom
90 is ascending towards left
180 ascending towards top
270 ascending towards right

